# Preprocessing

In [ ]:
import os
import io
import cv2
from PIL import Image
import h5py
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import preprocessing 

In [ ]:
projectDir = os.getcwd() + "/"
parentDir = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/"
dataPath = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/isic-2024-challenge/"
print("Data path path:", dataPath)
train_metadata = pd.read_csv(dataPath + "train-metadata.csv", sep=",")